### PROJET N° 7 - Implémenter un modèle de scoring

Auteur : Florian FOLLAIN
Date : Fevrier 2024

Plan

* [1 Importation des bibliothèques ](#chapter1)
* [2 Preprocessing des données ](#chapter2)
* [3 Création du dataframe ](#chapter3)
* [4 Dernier nettoyage  ](#chapter4)

#### <span style="color:blue">1 Importation des bibliothèques<a class="anchor" id="chapter1"></a></span>

In [1]:
import numpy as np
import pandas as pd
import psutil
import os
import gc
import re
import time
from contextlib import contextmanager
from sklearn.impute import SimpleImputer
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

In [2]:
def print_memory_usage():
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0]/(2.**30)
    print(f'mémoire RAM utilisée: {memory_use:.2f} GB')

In [3]:
def reduce_mem_usage(df):
    """ 
    Cette fonction réduit chaque type de données au format le plus léger       
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

#### Les 3 fonctions suivantes vont être utilisées pour nettoyer et encoder les données pendant le preprocessing

In [4]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

def clean_feature_names(df):
    def clean_name(name):
        return re.sub(r'[^A-Za-z0-9_]', '_', name)
    clean_names = [clean_name(col) for col in df.columns]
    df.columns = clean_names
    return df


# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

#### <span style="color:blue">2 Preprocessing des données<a class="anchor" id="chapter2"></a></span>

In [5]:
 # Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv(r'C:\cygwin64\openclassrooms\Projet_7\data\application_train.csv', nrows= num_rows)
    test_df = pd.read_csv(r'C:\cygwin64\openclassrooms\Projet_7\data\application_test.csv', nrows= num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']

    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)

    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    del test_df
    gc.collect()
    return df

# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv(r'C:\cygwin64\openclassrooms\Projet_7\data\bureau.csv', nrows = num_rows)
    bb = pd.read_csv(r'C:\cygwin64\openclassrooms\Projet_7\data\bureau_balance.csv', nrows = num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)

    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()

    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']

    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv(r'C:\cygwin64\openclassrooms\Projet_7\data\previous_application.csv', nrows = num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']

    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows = None, nan_as_category = True):
    pos = pd.read_csv(r'C:\cygwin64\openclassrooms\Projet_7\data\POS_CASH_balance.csv', nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']

    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg

# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv(r'C:\cygwin64\openclassrooms\Projet_7\data\installments_payments.csv', nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg

# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = pd.read_csv(r'C:\cygwin64\openclassrooms\Projet_7\data\credit_card_balance.csv', nrows = num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg



#### <span style="color:blue">3 Création du dataframe<a class="anchor" id="chapter3"></a></span>

In [6]:
print('RAM memory % used:', psutil.virtual_memory()[2])
print_memory_usage()

def main(debug = False):
    num_rows = 10000 if debug else None
    
    df = application_train_test(num_rows)
    bureau = bureau_and_balance(num_rows)
    print("Bureau df shape:", bureau.shape)
    df = df.join(bureau, how='left', on='SK_ID_CURR')
    del bureau
    gc.collect()
    
       
    
    prev = previous_applications(num_rows)
    print("Previous applications df shape:", prev.shape)
    df = df.join(prev, how='left', on='SK_ID_CURR')
    del prev
    gc.collect()
  
    pos = pos_cash(num_rows)
    print("Pos-cash balance df shape:", pos.shape)
    df = df.join(pos, how='left', on='SK_ID_CURR')
    del pos
    gc.collect()
    
    
    ins = installments_payments(num_rows)
    print("Installments payments df shape:", ins.shape)
    df = df.join(ins, how='left', on='SK_ID_CURR')
    del ins
    gc.collect()
    
    
    cc = credit_card_balance(num_rows)
    print("Credit card balance df shape:", cc.shape)
    df = df.join(cc, how='left', on='SK_ID_CURR')
    del cc
    gc.collect()
        
    
    df.to_csv("data.csv")
    
    return df

RAM memory % used: 56.7
mémoire RAM utilisée: 0.20 GB


In [7]:
data=main(debug = False)

Train samples: 307511, test samples: 48744
Bureau df shape: (305811, 116)
Previous applications df shape: (338857, 249)
Pos-cash balance df shape: (337252, 18)
Installments payments df shape: (339587, 26)
Credit card balance df shape: (103558, 141)


In [8]:
data = reduce_mem_usage(data)

Memory usage of dataframe is 1855.36 MB
Memory usage after optimization is: 651.98 MB
Decreased by 64.9%


In [9]:
data.shape

(356251, 798)

In [10]:

data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [11]:
ID = 'SK_ID_CURR'
TARGET = 'TARGET'
corr = data.drop(columns=[ID, TARGET]).corr().abs()

In [12]:
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
display(upper)

index  CODE_GENDER  FLAG_OWN_CAR  \
index                               NaN     0.003517      0.002091   
CODE_GENDER                         NaN          NaN      0.343445   
FLAG_OWN_CAR                        NaN          NaN           NaN   
FLAG_OWN_REALTY                     NaN          NaN           NaN   
CNT_CHILDREN                        NaN          NaN           NaN   
...                                 ...          ...           ...   
CC_NAME_CONTRACT_STATUS_nan_MAX     NaN          NaN           NaN   
CC_NAME_CONTRACT_STATUS_nan_MEAN    NaN          NaN           NaN   
CC_NAME_CONTRACT_STATUS_nan_SUM     NaN          NaN           NaN   
CC_NAME_CONTRACT_STATUS_nan_VAR     NaN          NaN           NaN   
CC_COUNT                            NaN          NaN           NaN   

                                  FLAG_OWN_REALTY  CNT_CHILDREN  \
index                                    0.001558      0.003617   
CODE_GENDER                              0.041584      0.048264   
FLAG_OWN_CAR                             0.002095      0.103048   
FLAG_OWN_REALTY                               NaN      0.001884   
CNT_CHILDREN                                  NaN           NaN   
...                                           ...           ...   
CC_NAME_CONTRACT_STATUS_nan_MAX               NaN           NaN   
CC_NAME_CONTRACT_STATUS_nan_MEAN              NaN           NaN   
CC_NAME_CONTRACT_STATUS_nan_SUM               NaN           NaN   
CC_NAME_CONTRACT_STATUS_nan_VAR               NaN           NaN   
CC_COUNT                                      NaN           NaN   

                                  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
index                                     0.008583    0.033421     0.025803   
CODE_GENDER                               0.078534    0.016903     0.076724   
FLAG_OWN_CAR                              0.089026    0.116993     0.143514   
FLAG_OWN_REALTY                           0.004257    0.034734     0.001622   
CNT_CHILDREN                              0.014069    0.005993     0.025994   
...                                            ...         ...          ...   
CC_NAME_CONTRACT_STATUS_nan_MAX                NaN         NaN          NaN   
CC_NAME_CONTRACT_STATUS_nan_MEAN               NaN         NaN          NaN   
CC_NAME_CONTRACT_STATUS_nan_SUM                NaN         NaN          NaN   
CC_NAME_CONTRACT_STATUS_nan_VAR                NaN         NaN          NaN   
CC_COUNT                                       NaN         NaN          NaN   

                                  AMT_GOODS_PRICE  REGION_POPULATION_RELATIVE  \
index                                    0.033626                    0.003424   
CODE_GENDER                              0.018228                    0.013555   
FLAG_OWN_CAR                             0.121237                    0.039089   
FLAG_OWN_REALTY                          0.042334                    0.018153   
CNT_CHILDREN                             0.002251                    0.024200   
...                                           ...                         ...   
CC_NAME_CONTRACT_STATUS_nan_MAX               NaN                         NaN   
CC_NAME_CONTRACT_STATUS_nan_MEAN              NaN                         NaN   
CC_NAME_CONTRACT_STATUS_nan_SUM               NaN                         NaN   
CC_NAME_CONTRACT_STATUS_nan_VAR               NaN                         NaN   
CC_COUNT                                      NaN                         NaN   

                                  DAYS_BIRTH  DAYS_EMPLOYED  \
index                               0.000021       0.006322   
CODE_GENDER                         0.151347       0.092361   
FLAG_OWN_CAR                        0.130522       0.022558   
FLAG_OWN_REALTY                     0.117180       0.026764   
CNT_CHILDREN                        0.329197       0.061678   
...                                      ...            ...   
CC_NAME_CONTRACT_STATUS_n

#### <span style="color:blue">4 CDerniers nettoyages<a class="anchor" id="chapter4"></a></span>

In [13]:
# find correlation better than 0.98
to_drop = [column for column in upper.columns if any(upper[column]>0.98)]
print(to_drop)
print(len(to_drop))

['AMT_GOODS_PRICE', 'YEARS_BUILD_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'OBS_60_CNT_SOCIAL_CIRCLE', 'NAME_CONTRACT_TYPE_Revolvingloans', 'NAME_INCOME_TYPE_Pensioner', 'ORGANIZATION_TYPE_XNA', 'BURO_MONTHS_BALANCE_MIN_MIN', 'BURO_CREDIT_ACTIVE_Closed_MEAN', 'ACTIVE_AMT_CREDIT_SUM_DEBT_MAX', 'ACTIVE_AMT_CREDIT_SUM_DEBT_SUM', 'ACTIVE_MONTHS_BALANCE_MIN_MIN', 'CLOSED_AMT_CREDIT_SUM_DEBT_SUM', 'CLOSED_MONTHS_BALANCE_MIN_MIN', 'PREV_AMT_CREDIT_MAX', 'PREV_AMT_GOODS_PRICE_MAX', 'PREV_FLAG_LAST_APPL_PER_CONTRACT_Y_MEAN', 'PREV_NAME_CASH_LOAN_PURPOSE_XAP_MEAN', 'PREV_NAME_PAYMENT_TYPE_XNA_MEAN', 'PREV_CODE_REJECT_REASON_CLIENT_MEAN', 'PREV_NAME_GOODS_CATEGORY_XNA_MEAN', 'PREV_PRODUCT_COMBINATION_nan_ME

#### Nous supprimons ici les variables qui présentent des corélations supérieurs à 98% ce qui permet de réduire le df de 89 colonnes

In [14]:
#drop them
data = data.drop(columns=to_drop)

In [15]:
data.shape

(356251, 709)

#### Nous remplaçons les valeurs infinies par des nan et nous les remplaçons par les médianes

In [16]:
nb_inf = np.isinf(data).values.sum()
print("Infinity values = {} ({:.2%})".format(nb_inf, nb_inf/data.size))
data.replace([np.inf, -np.inf], np.nan, inplace=True)

Infinity values = 21 (0.00%)


In [17]:
nan_values = data.isna().sum().sum()
print("Missing values = {} ({:.2%})".format(nan_values, nan_values/data.size))

Missing values = 56090587 (22.21%)


In [18]:
imp_mean = SimpleImputer(missing_values = np.NaN, strategy = 'mean')
    
data_imp = imp_mean.fit_transform(data)
print('RAM memory % used:', psutil.virtual_memory()[2])
print_memory_usage()

RAM memory % used: 67.2
mémoire RAM utilisée: 2.44 GB


In [19]:
data_df = pd.DataFrame(data_imp, columns=data.columns)

In [21]:
df = data_df.copy()
train_df = df[df['TARGET'].isin([0, 1])]
# Filtrer pour obtenir les lignes où 'TARGET' n'est ni 0 ni 1 pour le test_df
test_df = df[~df['TARGET'].isin([0, 1])]
    
print("Forme initiale de train_df: {}, forme de test_df: {}".format(train_df.shape, test_df.shape))

Forme initiale de train_df: (307507, 709), forme de test_df: (48744, 709)


#### Le train set et le test set sont prêts 

In [22]:
train_df.to_csv('train_df.csv')
test_df.to_csv('test_df.csv')